# **Extraction of Spanish Citizen Science Projects**
Web page: https://ciencia-ciudadana.es/proyecto-cc/ 

(Observatorio de la Ciencia Ciudadana en España)

In [1]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd

In [2]:
# Create an empty DataFrame with specified columns
df1 = pd.DataFrame(columns=['Project Name', 'Project Link', 'Project Scope', 'Project Goal', 'Project Description', 'Project Entity/Scientist', 'How To Join',
                           'Necessary Equipment', 'Initial Date', 'Final Date', 'Public Type', 'Location (Province)', 'Number of Participants', 'Results', 
                           'Link to Results', 'Project Impact', 'Why Using CC?', 'Citizen Science Web Name', 'Citizen Science Web Link'])

In [3]:
url = "https://ciencia-ciudadana.es/proyecto-cc/"
web_name = 'Observatorio de la Ciencia Ciudadana en España'

# Agregar encabezados a la solicitud
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'}

links = []
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, "html.parser")

# Find all elements with class name "underline"
underline_elements = soup.find_all('img', {'decoding': 'async'})

# Extract the links from the parent elements
for element in underline_elements:
  parent_a_tag = element.find_parent('a')
  if parent_a_tag and 'href' in parent_a_tag.attrs:
    link = parent_a_tag['href']
    links.append(link)

In [4]:
len(links)

345

In [8]:
def get_complete_section(proj_soup, dtbf):
  proj_segment = proj_soup.find_all('div', {'class': 'tb-field', 'data-toolset-blocks-field': dtbf})
  return ''.join([seg.text for seg in proj_segment])

In [9]:
def get_project_info1(project_link, main_url, main_name):
  proj_info = ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']

  response = requests.get(project_link, headers=headers)
  proj_soup = BeautifulSoup(response.content, "html.parser")
  proj_title = proj_soup.find('h1', {'class': 'entry-title'}).text
  proj_scope = get_complete_section(proj_soup, "28515d8ce1fa37a6527af15754983e83")
  proj_goal = get_complete_section(proj_soup, "ba74407e8b20cac888e283e8576140f9")
  proj_desc = get_complete_section(proj_soup, "7147dde37d9b86b4d4a2dc89b9c12945")
  proj_entity = get_complete_section(proj_soup, "486908e9509d0a139e234cbfb7f8f47d")
  proj_join = get_complete_section(proj_soup, "687686b4b9d6f02068b541e6bc6f2812")
  proj_equip = get_complete_section(proj_soup, "cbcd66c16990127ea3c06cebc87c17f0")
  proj_ini = get_complete_section(proj_soup, "3d66c4d00ab960c97a58ac752fe406f4")
  proj_end = get_complete_section(proj_soup, "9a00b649618c1cd0c0cedd6cfc386f09")
  proj_public = get_complete_section(proj_soup, "e9e0eef27b43baa290bdc058d0ec6cee")
  proj_loc = get_complete_section(proj_soup, "05399f19f4997ca4dc2a1f1770db2d80")
  proj_amt_part = get_complete_section(proj_soup, "5ee97fadc3c34313fff123d0cba66a1a")
  proj_results = get_complete_section(proj_soup, "b064a4637177bd64d367dc70864be32c")

  proj_link_res = proj_soup.find_all('div', {'class': 'tb-field', 'data-toolset-blocks-field': "c5d8b6598ffd2835065c71792c189772"})
  proj_link_res = [seg.text for seg in proj_link_res][0]

  proj_impact = get_complete_section(proj_soup, "4297f88ef1601b65151fcf8fc6fccadd")
  proj_useCC = get_complete_section(proj_soup, "974e65ee4de5d532951d366fa43dc400")

  # Create a dictionary with the values for the new row
  new_row = {
  'Project Name': proj_title,
  'Project Link': project_link,
  'Project Scope': proj_scope,
  'Project Goal': proj_goal,
  'Project Description': proj_desc,
  'Project Entity/Scientist': proj_entity,
  'How To Join': proj_join,
  'Necessary Equipment': proj_equip,
  'Initial Date': proj_ini,
  'Final Date': proj_end,
  'Public Type': proj_public,
  'Location (Province)': proj_loc,
  'Number of Particpiants': proj_amt_part,
  'Results': proj_results,
  'Link to Results': proj_link_res,
  'Project Impact': proj_impact,
  'Why Using CC?': proj_useCC,
  'Citizen Science Web Name': main_name,
  'Citizen Science Web Link': main_url
  }

  # Add the new row to the DataFrame using the loc indexer
  df1.loc[len(df1)] = new_row

In [10]:
for link in links:
  get_project_info1(link, url, web_name)

In [11]:
df1

,Project Name,Project Link,Project Scope,Project Goal,Project Description,Project Entity/Scientist,How To Join,Necessary Equipment,Initial Date,Final Date,Public Type,Location (Province),Number of Participants,Results,Link to Results,Project Impact,Why Using CC?,Citizen Science Web Name,Citizen Science Web Link
0,LADA-UC3M,https://ciencia-ciudadana.es/proyecto-cc/lada-...,Cultura y Arqueología,Nuestro laboratorio pretende integrar a la com...,Es un laboratorio de ciencia ciudadana en el q...,"Departamento de Humanidades: Historia, Geograf...",Todo el mundo es bienvenido\nNo es necesaria i...,Es necesario que cada voluntario traiga su pro...,4 de octubre de 2021,23 de marzo de 2050,Todo tipo de público,Madrid,NaN,,,"científico, educativo, tecnológico",La ciencia ciudadana aplicada en arqueología p...,Observatorio de la Ciencia Ciudadana en España,https://ciencia-ciudadana.es/proyecto-cc/
1,PAAM,https://ciencia-ciudadana.es/proyecto-cc/paam/,"Ciencias Sociales, Educación, Informática y Ci...",El objetivo es llegar al máximo número de pers...,"El proyecto programa PAAM, tiene dos fases:\nL...",Carlos Sequeiro Lorenzo (ONG Labecos),Personas de este perfil:\nPsicólogos/as\nPsicó...,No se precisa de un equipamiento especial a ni...,10 de abril de 2023,2 de abril de 2024,Mayores (mayores de 65 años),Vigo,NaN,Los resultados esperados son una mejora en la ...,,"científico, económico, social, educativo, polí...","Ocuparse de asuntos necesarios, condenados al ...",Observatorio de la Ciencia Ciudadana en España,https://ciencia-ciudadana.es/proyecto-cc/
2,Cities at Night,https://ciencia-ciudadana.es/proyecto-cc/citie...,"Biodiversidad, Ecología y Medioambiente, Espac...","A nivel mundial, la luz artificial nocturna (A...",Cities at Night nace en 2014 liderada por el d...,Responsable del Proyecto: Alejandro Sánchez de...,Puedes participar en el programa como:\nVolunt...,Si quieres participar sobre las imágenes noctu...,13 de febrero de 2014,31 de enero de 2050,Todo tipo de público,Madrid,NaN,Durante el primer año del programa obtuvimos l...,Papers Científicos:\n1.- Environmental risks f...,"científico, educativo, medioambiental",Hacemos ciencia ciudadana porque sin ellos ser...,Observatorio de la Ciencia Ciudadana en España,https://ciencia-ciudadana.es/proyecto-cc/
3,FuenAragón,https://ciencia-ciudadana.es/proyecto-cc/fuena...,"Biodiversidad, Ciencias de la Agricultura y Ve...",El objetivo de FuenAragón es elaborar un mapa ...,El desarrollo del proyecto contaré con varias ...,"Fundación Ibercivis, CITA (Centro de Investiga...",Para participar en FuenAragón hay que seleccio...,Es necesario un smartphone Android para descar...,1 de julio de 2022,30 de junio de 2023,Todo tipo de público,Zaragoza,NaN,Los resultados del proyecto se conocerán a par...,,"científico, económico, social, educativo, medi...",La ciencia ciudadana es una forma de democrati...,Observatorio de la Ciencia Ciudadana en España,https://ciencia-ciudadana.es/proyecto-cc/
4,Laboratorio Ciudadano de Salud Urbana,https://ciencia-ciudadana.es/proyecto-cc/labor...,"Biodiversidad, Ecología y Medioambiente, Medic...",El Laboratorio de Salud Urbana es un espacio c...,La ciudadanía tiene un rol fundamental a la ho...,Instituto de Salud Global y LICHEN Innovación ...,,,1 de enero de 2022,31 de diciembre de 2022,Todo tipo de público,Barcelona,NaN,Del proceso de ciencia ciudadana se ha obtenid...,https://www.labcsu.com/recursos/\n,"científico, social, educativo, medioambiental,...",,Observatorio de la Ciencia Ciudadana en España,https://ciencia-ciudadana.es/proyecto-cc/
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340,ÓSCAR DÍAZ,https://ciencia-ciudadana.es/proyecto-cc/oscar...,,Colaboré en un proyecto para evaluar la felici...,,USJ,,,,,Todo tipo de público,Zaragoza,NaN,http://www.aragondigital.es/noticia.asp?notid=...,http://www.ibercivis.es/la-nueva-aplicacion-ha...,social,,Observatorio de la Ciencia Ciudadana en España,https://ciencia-ciudadana.es/proy

In [12]:
from google.colab import files

df1.to_csv('/content/Spanish_CS_Projects.csv', index=False)
files.download('/content/Spanish_CS_Projects.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [13]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 345 entries, 0 to 344
Data columns (total 19 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Project Name              345 non-null    object 
 1   Project Link              345 non-null    object 
 2   Project Scope             345 non-null    object 
 3   Project Goal              345 non-null    object 
 4   Project Description       345 non-null    object 
 5   Project Entity/Scientist  345 non-null    object 
 6   How To Join               345 non-null    object 
 7   Necessary Equipment       345 non-null    object 
 8   Initial Date              345 non-null    object 
 9   Final Date                345 non-null    object 
 10  Public Type               345 non-null    object 
 11  Location (Province)       345 non-null    object 
 12  Number of Participants    0 non-null      float64
 13  Results                   345 non-null    object 
 14  Link to Re

# **Extraction of Catalan Citizen Science Projects**
Web page: https://www.barcelona.cat/barcelonaciencia/es/ciencia-en-la-ciudad/la-ciencia-y-la-ciudadania/ciencia-ciudadana 

(CS Projects from Barcelona)

In [14]:
df2 = pd.DataFrame(columns=['Project Name', 'Project Link', 'Project Description', 'Project Info', 'Project State', 'Link to Project Data', 
                           'Activities within the framework of the Office', 'Project Scope', 'Citizen Science Web Name', 'Citizen Science Web Link'])

In [15]:
url = "https://www.barcelona.cat/barcelonaciencia/es/ciencia-en-la-ciudad/la-ciencia-y-la-ciudadania/ciencia-ciudadana"
web_name = 'Ciencia Ciudadana Ayuntamiento de Barcelona'

# Agregar encabezados a la solicitud
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'}

links = []
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, "html.parser")

# Find all elements with class name "underline"
underline_elements = soup.find_all('a', {'class': 'item-link'})

# Extract the links from the parent elements
for element in underline_elements:
    link = element['href']
    links.append('https://www.barcelona.cat'+link)

In [16]:
def get_segments(text):
  state = ''
  desc = ''
  activities = ''
  scope = ''
  results = ''

  for i in text:
    if i.startswith('Estado'):
      state = i.replace('Estado:', '')

    elif i.startswith('Actividades en el marco de la Oficina:'):
      activities = i.replace('Actividades en el marco de la Oficina:', '')

    elif i.startswith('Ámbito:'):
      scope = i.replace('Ámbito:', '')

    elif i.startswith('Dónde visualizar los datos recogidos:'):
      results = i.replace('Dónde visualizar los datos recogidos:', '')

    elif i == '':
      continue

    else:
      desc = desc + i

  return desc, state, activities, scope, results

In [17]:
def get_project_info2(project_link, main_url, main_name):

    response = requests.get(project_link, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")

    proj_name = soup.find('h2', {'class': 'page-title'})
    if proj_name is not None:
      proj_name = proj_name.text.replace('\n  ', '').replace('\n', '')
    else:
      print("Project Name not found or is None")

    proj_info = soup.find('div', {'class': "field__item entradeta"})
    if proj_info is not None:
      proj_info = proj_info.text.replace('\n  ', '').replace('\n', '')
    else:
      print("Project Info not found or is None")

    proj_desc = soup.find('div', {'class': "field field--name-body field--type-text-with-summary field--label-hidden"})
    elements = proj_desc.find_all('p')
    text = [p.get_text(strip=True) for p in elements]

    desc, state, activities, scope, results = get_segments(text)
    

    # Create a dictionary with the values for the new row
    new_row = {
    'Project Name': proj_name,
    'Project Link': project_link,
    'Project Description': proj_info,
    'Project Info': desc,
    'Project State': state,
    'Link to Project Data': results,
    'Activities within the framework of the Office': activities,
    'Project Scope': scope,
    'Citizen Science Web Name': main_name,
    'Citizen Science Web Link': main_url
    }

    # Add the new row to the DataFrame using the loc indexer
    df2.loc[len(df2)] = new_row

In [18]:
for link in links:
  get_project_info2(link, url, web_name)

In [19]:
df2

,Project Name,Project Link,Project Description,Project Info,Project State,Link to Project Data,Activities within the framework of the Office,Project Scope,Citizen Science Web Name,Citizen Science Web Link
0,Beepath,https://www.barcelona.cat/barcelonaciencia/es/...,Herramienta que permite estudiar la movilidad ...,"Se realiza con la participación directa, volun...",activo periódicamente.,,"Programa en los Barrios, Programa en las escue...",social.,Ciencia Ciudadana Ayuntamiento de Barcelona,https://www.barcelona.cat/barcelonaciencia/es/...
1,BioBlitz Barris,https://www.barcelona.cat/barcelonaciencia/es/...,BioBlitz Barris pretende utilizar el método Bi...,El objetivo es dar a conocer y valorar la biod...,activo periódicamente.,BioBlitz Barris.,"Programa en los Barrios, Fiesta de la Ciencia,...",ambiental.,Ciencia Ciudadana Ayuntamiento de Barcelona,https://www.barcelona.cat/barcelonaciencia/es/...
2,BioBlitzBcn,https://www.barcelona.cat/barcelonaciencia/es/...,Un BioBlitz es un censo exhaustivo y colaborat...,"Típicamente realizado en parques urbanos, esta...",activo periódicamente.,Museo de Ciencias Naturales de Barcelona.,"Programa en los Barrios, Comunidad de práctica.",ambiental.,Ciencia Ciudadana Ayuntamiento de Barcelona,https://www.barcelona.cat/barcelonaciencia/es/...
3,Cities-Health,https://www.barcelona.cat/barcelonaciencia/es/...,El proyecto busca situar las preocupaciones de...,"Residentes de cinco ciudades europeas, entre e...",activo.,Cities-Health.,"Fiesta de la Ciencia, Comunidad de práctica.",salud.,Ciencia Ciudadana Ayuntamiento de Barcelona,https://www.barcelona.cat/barcelonaciencia/es/...
4,Floodup,https://www.barcelona.cat/barcelonaciencia/es/...,Floodup tiene como objetivos mejorar el conoci...,"Para facilitar la participación, se han creado...",activo.,,"Programa en las escuelas, Fiesta de la Ciencia...",ambiental.,Ciencia Ciudadana Ayuntamiento de Barcelona,https://www.barcelona.cat/barcelonaciencia/es/...
5,Genigma,https://www.barcelona.cat/barcelonaciencia/es/...,Este proyecto codiseña con la ciudadanía un ju...,El juegoGenigmaquiere ser una herramienta medi...,activo.,Genigma Game.,Comunidad de práctica.,salud.,Ciencia Ciudadana Ayuntamiento de Barcelona,https://www.barcelona.cat/barcelonaciencia/es/...
6,InSPIRES,https://www.barcelona.cat/barcelonaciencia/es/...,El proyecto InSPIRES apunta al levantamiento r...,El proyecto colabora con la Oficina en los pro...,,Inspires.,,social.,Ciencia Ciudadana Ayuntamiento de Barcelona,https://www.barcelona.cat/barcelonaciencia/es/...
7,Juegos para el Cambio Social,https://www.barcelona.cat/barcelonaciencia/es/...,El proyecto recoge experimentos portátiles dir...,Utiliza dilemas sociales en forma de juegos pa...,,,"Fiesta de la Ciencia, Bienal de Ciencia 2018, ...",social.,Ciencia Ciudadana Ayuntamiento de Barcelona,https://www.barcelona.cat/barcelonaciencia/es/...
8,Líquenes de Barcelona,https://www.barcelona.cat/barcelonaciencia/es/...,Los líquenes son organismos singulares y muy r...,Son muy sensibles al aumento de concentracione...,activo periódicamente.,Natusfera.,"Programa en los Barrios, Fiesta de la Ciencia,...",ambiental.,Ciencia Ciudadana Ayuntamiento de Barcelona,https://www.barcelona.cat/barcelonaciencia/es/...
9,Mapa Sonoro de Barcelona,https://www.barcelona.cat/barcelonaciencia/es/...,"En cada barrio, Mapa Sonoro de Barcelona se en...","En este proceso, la ciudadanía participa en di...",activo.,Instamaps.,"Programa en los Barrios, Programa en las escue...","ambiental, salud.",Ciencia Ciudadana Ayuntamiento de Barcelona,https://www.barcelona.cat/barcelonaciencia/es/...


In [20]:
from google.colab import files

df2.to_csv('/content/Catalan_CS_Projects.csv', index=False)
files.download('/content/Catalan_CS_Projects.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [21]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22 entries, 0 to 21
Data columns (total 10 columns):
 #   Column                                         Non-Null Count  Dtype 
---  ------                                         --------------  ----- 
 0   Project Name                                   22 non-null     object
 1   Project Link                                   22 non-null     object
 2   Project Description                            22 non-null     object
 3   Project Info                                   22 non-null     object
 4   Project State                                  22 non-null     object
 5   Link to Project Data                           22 non-null     object
 6   Activities within the framework of the Office  22 non-null     object
 7   Project Scope                                  22 non-null     object
 8   Citizen Science Web Name                       22 non-null     object
 9   Citizen Science Web Link                       22 non-null     obje

# **Extraction Example**

In [22]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd

In [23]:
# Create an empty DataFrame with specified columns
df3 = pd.DataFrame(columns=['Project Name', 'Project Link', 'Objective', 'Principal Scientist',
                           'Scientist Affiliation', 'Dates', 'Location', 'Project Type', 'Cost', 'Grade Level',
                           'Time Commitment', 'How To Join', 'Description', 'Citizen Science Web Name',
                           'Citizen Science Web Link'])

In [24]:
url = "https://www.scientificamerican.com/citizen-science/"
web_name = 'Scientific American'

# Agregar encabezados a la solicitud
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'}

# projects = []
links = []

# Hemos visto que hay 12 páginas de proyectos: loop para acceder a las 12 págs.
for i in range(1, 13):
    page_url = url + "?page=" + str(i)

    # Realizar la petición a la página web con los encabezados
    response = requests.get(page_url, headers=headers)

    soup = BeautifulSoup(response.content, "html.parser")
    
    '''
    project_titles = soup.find_all('span', {'class': 'underline'})
    project_titles2 = [title.text for title in project_titles]
    projects.extend(project_titles2)
    '''

    # Find all elements with class name "underline"
    underline_elements = soup.find_all('span', {'class': 'underline'})

    # Extract the links from the parent elements
    for element in underline_elements:
        parent_a_tag = element.find_parent('a')
        if parent_a_tag and 'href' in parent_a_tag.attrs:
            link = parent_a_tag['href']
            links.append(link)

In [25]:
# Imprimir el código HTML de la página
# print(soup.prettify())

In [26]:
def get_info_details(proj_info):
    if proj_info.lower().startswith('principal scientist'):
        return 0
    elif proj_info.lower().startswith('scientist affiliation'):
        return 1
    elif proj_info.lower().startswith('dates'):
        return 2
    elif proj_info.lower().startswith('location'):
        return 3
    elif proj_info.lower().startswith('project type'):
        return 4
    elif proj_info.lower().startswith('cost'):
        return 5
    elif proj_info.lower().startswith('grade level'):
        return 6
    elif proj_info.lower().startswith('time commitment'):
        return 7
    elif proj_info.lower().startswith('how to join'):
        return 8

In [27]:
def get_project_info3(project_link, main_url, main_name):
    proj_info = ['', '', '', '', '', '', '', '', '']

    response = requests.get(project_link, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
    proj_title = soup.find('h1', {'class': 'article-header__title t_article-title'}).text
    proj_obj = soup.find('p').text.replace("\n", "").replace("\xa0", "")

    proj_info2 = soup.find_all('li', {'class': 'citizen-science-details__list__item'})
    proj_info2 = [info.text for info in proj_info2]
    for detail in proj_info2:
        idx = get_info_details(detail)
        if idx == 0:
            proj_info[0] = detail.replace("Principal Scientist:  ", "")
        elif idx == 1:
            proj_info[1] = detail.replace("Scientist Affiliation:  ", "")
        elif idx == 2:
            proj_info[2] = detail.replace("Dates:  ", "")
        elif idx == 3:
            proj_info[3] = detail.replace("LOCATION:  ", "")
        elif idx == 4:
            proj_info[4] = detail.replace("PROJECT TYPE:  ", "")
        elif idx == 5:
            proj_info[5] = detail.replace("COST:  ", "")
        elif idx == 6:
            proj_info[6] = detail.replace("GRADE LEVEL:  ", "")
        elif idx == 7:
            proj_info[7] = detail.replace("TIME COMMITMENT:  ", "")
        elif idx == 8:
            proj_info[8] = detail.replace("HOW TO JOIN:  ", "")
    
    proj_desc = soup.find_all('div', {'class': 'article-block article-text'})
    proj_desc = [info.text.replace("\n", "").replace("\xa0", "") for info in proj_desc][0]

    # Create a dictionary with the values for the new row
    new_row = {
    'Project Name': proj_title,
    'Project Link': project_link,
    'Objective': proj_obj,
    'Principal Scientist': proj_info[0],
    'Scientist Affiliation': proj_info[1],
    'Dates': proj_info[2],
    'Location': proj_info[3],
    'Project Type': proj_info[4],
    'Cost': proj_info[5],
    'Grade Level': proj_info[6],
    'Time Commitment': proj_info[7],
    'How To Join': proj_info[8],
    'Description': proj_desc,
    'Citizen Science Web Name': main_name,
    'Citizen Science Web Link': main_url
    }

    # Add the new row to the DataFrame using the loc indexer
    df3.loc[len(df3)] = new_row

In [28]:
for link in links:
    get_project_info3(link, url, web_name)

In [29]:
df3

,Project Name,Project Link,Objective,Principal Scientist,Scientist Affiliation,Dates,Location,Project Type,Cost,Grade Level,Time Commitment,How To Join,Description,Citizen Science Web Name,Citizen Science Web Link
0,Caribbean Storms 2017,https://www.scientificamerican.com/citizen-sci...,Analyze satellite images of areas stricken by ...,N/A,N/A,Ongoing,,Observation,Free,All Ages,Variable,Visit The Planetary Response Network to get st...,The Zooniverse has been asked to enableThe Pla...,Scientific American,https://www.scientificamerican.com/citizen-sci...
1,Zooniverse: Wildwatch Kenya,https://www.scientificamerican.com/citizen-sci...,"Help researchers count, identify and track end...",David O'Connor,Researcher in Global Partnerships at the Insti...,Ongoing,,Data Processing,Free,All Ages,Variable,Register with Zooniverse and proceed to the Wi...,"Northern Kenya is a vast, varied and breathtak...",Scientific American,https://www.scientificamerican.com/citizen-sci...
2,The Plastic Tide,https://www.scientificamerican.com/citizen-sci...,Help scientists figure out where the millions ...,"Peter Kohler, founder and director of The Plas...",The Scientific Exploration Society and the Roy...,Ongoing,,Data Processing,Free,All Ages,Variable,Register at the Zooniverse Web site. Then use ...,Estimates are currently at trillions of pieces...,Scientific American,https://www.scientificamerican.com/citizen-sci...
3,Citizen Endo,https://www.scientificamerican.com/citizen-sci...,Help researchers better understand endometriosis,"Noemie Elhadad, Associate Professor",Columbia University Department of Biomedical I...,Ongoing,,Questionnaire,Free,High/Secondary School,More than 5 hours,"To download the Phendo research app, visit Phe...",The Citizen Endo project is an active collabor...,Scientific American,https://www.scientificamerican.com/citizen-sci...
4,Small World of Words,https://www.scientificamerican.com/citizen-sci...,Help researchers discover how the meaning of w...,Simon De Deyne/Gert Storms,"University of Adelaide, Australia/ University ...",Ongoing,,Questionnaire,Free,All Ages,Variable,"To participate, visit the Small World of Words...",The Small World of Words project is a large-sc...,Scientific American,https://www.scientificamerican.com/citizen-sci...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,Participatory Urban Sensing,https://www.scientificamerican.com/citizen-sci...,Through the use of sensors built into your mob...,Deborah Estrin,"Center for Embedded Networked Sensing, Univers...",Ongoing,- Urban settings,Fieldwork,Free,All Ages,Variable,"Contact Urban Sensing CENS/UCLA via e-mail, te...",Participatory Urban Sensing emphasizes the inv...,Scientific American,https://www.scientificamerican.com/citizen-sci...
236,Scuttle Fly Study,https://www.scientificamerican.com/citizen-sci...,Researchers want your help collecting scuttle ...,Suzanne McGaugh,Duke University Center for Theoretical & Mathe...,Ongoing,,Fieldwork,Free,All Ages,Less than 1 hour per week,"We will send participants collection vials, de...",Researchers at Duke University's Noor lab of E...,Scientific American,https://www.scientificamerican.com/citizen-sci...
237,Zooniverse: Planet Hunters,https://www.scientificamerican.com/citizen-sci...,Use data from NASA's Kepler spacecraft to find...,Debra Fischer,"Yale University, San Francisco State University",Ongoing,,Data Processing,Free,All Ages,Variable,Register at the Zooniverse Web site.\n\r\nFor ...,"Since 1995, more than 500 planets have been di...",Scientific American,https://www.scientificamerican.com/citizen-sci...
238,Foldit Online Protein Puzzle,https://www.scientificamerican.com/citizen-sci...,An online game that has players solve 3-D puzz...,David Baker,University of Washington at Seattle,Ongoing,,Data Processing,Free,All Ages,Variable,Download Foldit software from the game's Web s...,"Inside your cells, proteins allow your body to...",Scientific American,https://www.scientificamerican.com/citizen-s

In [30]:
df3['Project Link'].iloc[239]

'https://www.scientificamerican.com/citizen-science/depts-washington-edu-natmap/'

In [31]:
from google.colab import files

df3.to_csv('/content/Scientific_American_Projects.csv', index=False)
files.download('/content/Scientific_American_Projects.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [32]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 240 entries, 0 to 239
Data columns (total 15 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Project Name              240 non-null    object
 1   Project Link              240 non-null    object
 2   Objective                 240 non-null    object
 3   Principal Scientist       240 non-null    object
 4   Scientist Affiliation     240 non-null    object
 5   Dates                     240 non-null    object
 6   Location                  240 non-null    object
 7   Project Type              240 non-null    object
 8   Cost                      240 non-null    object
 9   Grade Level               240 non-null    object
 10  Time Commitment           240 non-null    object
 11  How To Join               240 non-null    object
 12  Description               240 non-null    object
 13  Citizen Science Web Name  240 non-null    object
 14  Citizen Science Web Link  